# 3.1 Operators based on first derivative

Edge detection is an image processing technique for finding those points in images that represent brightness variations or *edges*. It has many applications for image segmentation and data extraction that will be explained posteriorly in this course.

As commented, edges can be defined as transitions between image regions that have different gray levels (intensities). In this way, the unidimensional, continuous model of an ideal edge is:

<img src="./images/model_of_ideal_edge.PNG" width="600">

However, in real images that model is not exactly followed by edges since images are discrete, and they are corrupted by noise.

Moreover, **the nature of edges may be diverse**:
- occlusion borders
- different orientation of surfaces
- different reflectance properties
- different textures
- illumination effects: shadows, reflections, etc.

### Error types related to edge detection

Finding edges properly is not a straightforward task, as there exist different errors that can appear when applying edge detection techniques:
- **Detection error.** A good detector exhibits low ratio of false negative and false positive, that is:
  - False negatives: Existing edges that are not detected.
  - False positives: Detected objects that are not real.
- **Localization error.** Edges are detected, but they are not at the real, exact position.
- **Multiple response.** Multiple detections are raised for the same edge (the edge is thick).

The following figure illustrates such errors.

<img src="./images/edge_errors.jpg" width="400">

Thereby, when designing a good edge detector, the goal is to achieve low detection and localization errors, as well as to avoid multiple responses. 

### Operators based on first derivative (gradient)

In the upcoming chapters, we are going to investigate and implement different edge detection methods. All of them are based on convolution, having their own pros and cons.

Concretely, in this notebook we will cover **first-derivative** based operators, which try to detect borders by looking at abrupt intensity differences in neighbor pixels. In the image below we can see two functions $f(x)$ (first row) and how their derivatives (second row) reach their maximum values at the points where the functions' values change more abruptly (around $x_o$). 

<img src="./images/first_derivative_of_continuous_function.PNG" width="500">$\\[10pt]$

If we are dealing with a **two-dimensional** continuous function $f(x,y)$, its derivative is a *vector* (**gradient**) defined as:$\\[10pt]$
$$
\nabla  f(x,y) = 
\begin{bmatrix} 
\frac{\partial}{\partial x}f(x,y) \\ \frac{\partial}{\partial y}f(x,y) 
\end{bmatrix} 
=
\begin{bmatrix}
f_x(x,y) \\
f_y(x,y)
\end{bmatrix}
$$

which points at the *direction* of maximum (positive) variation of $f(x,y)$:$\\[10pt]$

$$
\alpha(x,y) = \arctan \left(\frac{f_y(x,y)}{f_x(x,y)}\right)
$$

and has a *module* proportional to the strength of this variation:$\\[10pt]$

$$
|\nabla  f(x,y)| = \sqrt{(f_x(x,y))^2 + (f_y(x,y))^2} \approx |f_x(x,y)| + |f_y(x,y)|
$$

The image below shows examples of gradient vectors:$\\[10pt]$

<img src="./images/gradient_examples.PNG" width="700">$\\[10pt]$

Concretely, the techniques based on the first derivative explored here are:

- Discrete approximations of a **gradient operator** (Sobel, Prewitt, Roberts, etc., Section 3.1.1).
- The **Derivative of Gaussian** (DroG) operator (Section 3.1.2).

## Problem context - Edge detection for medical images

Edge detection in medical images is of capital importance for diagnosis of different diseases (e.g., the detection of tumor cells) in human organs such as lungs and prostates, becoming an essential pre-processing step in medical image segmentation.

<img src="./images/hospital_clinico.jpg" width="400">

In this context, *Hospital Clínico*, a very busy hospital in Málaga is asking local engineering students to join their research team. They are looking for a person with knowledge in image processing and, in order to ensure it, they have published 3 medical images: `medical_1.jpg`, `medical_2.jpg` and `medical_3.jpg`. They have asked us to perform accurate edge detection in the three images, as well as to provide an explanation of how it has been made.

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib
from ipywidgets import interactive, fixed, widgets
matplotlib.rcParams['figure.figsize'] = (15.0, 15.0)

images_path = './images/'

To face this challenge, we are going to use plenty edge detection methods, which will be tested and compared in order to determine the best option.

### **<span style="color:green"><b><i>ASSIGNMENT 1: Taking a look at images</i></b></span>**

First, **display the provided images** to get an idea about what we are dealing with.

*Note: As most medical images does not provide color information, we are going to use border detection in greyscale images.*

*Tip: Different approaches can be followed for edge detection in color images, like converting to YCrCb color space (appendix 2), or detecting edges on each RGB channel.*

In [ ]:
# ASSIGNMENT 1
# Display the provided images in a 1x3 plot to see what are we dealing with
# Write your code here!

# Read the images


# And show them


## 3.1.1 Discrete approximations of a gradient operator

The first bunch of methods that we are going to explore carry out a **discrete approximation of a gradient operator** based on the differences between gray (intensity) levels. For example:

<div style="padding: 10px; margin: 0 auto; text-align: center; display: table;">

<div style = "padding: 0px; display: table-cell; vertical-align: middle; width: 370px; text-align:left;">- Backward difference of pixels along a row: <br /> $\, \, \, \,f_x(x,y) \approx G_C(i,j)=[F(i,j)-F(i-1,j)]/T$ </div>
    
<table style = "display: table-cell; vertical-align: middle;">
	<tbody>
		<tr>
			<td style="border: 1px solid;">0</td>
			<td style="border: 1px solid;">0</td>
			<td style="border: 1px solid;">0</td>
		</tr>
		<tr>
			<td style="border: 1px solid;">0</td>
			<td style="border: 1px solid;" bgcolor="#81F7F3">1</td>
			<td style="border: 1px solid;">-1</td>
		</tr>
		<tr>
			<td style="border: 1px solid;">0</td>
			<td style="border: 1px solid;">0</td>
			<td style="border: 1px solid;">0</td>
		</tr>
	</tbody>
</table> 
    
</div>

<div style="padding: 10px; margin: 0 auto; text-align: center; display: table;">

<div style = "padding: 0px; display: table-cell; vertical-align: middle; width: 370px; text-align:left;">- Symmetric difference of pixels along a row: <br /> $\, \, \, \,f_x(x,y) \approx G_R(i,j)=[F(i+1,j)-F(i-1,j)]/2T$ </div>
    
<table style = "display: table-cell; vertical-align: middle;">
	<tbody>
		<tr>
			<td style="border: 1px solid;">0</td>
			<td style="border: 1px solid;">0</td>
			<td style="border: 1px solid;">0</td>
		</tr>
		<tr>
			<td style="border: 1px solid;">1</td>
			<td style="border: 1px solid;" bgcolor="#81F7F3">0</td>
			<td style="border: 1px solid;">-1</td>
		</tr>
		<tr>
			<td style="border: 1px solid;">0</td>
			<td style="border: 1px solid;">0</td>
			<td style="border: 1px solid;">0</td>
		</tr>
	</tbody>
</table>     

</div>

These approximations are typically implemented through the convolution of the image with a pair of templates $H_C$ (for columns, detecting vertical edges) and $H_R$ (for rows, detecting horizontal ones), that is:

$$
G_R(i,j) = F(i,j) \otimes H_R(i,j) \\
G_C(i,j) = F(i,j) \otimes H_C(i,j)
$$

Perhaps the most popular operator doing this is such of **Sobel**, although there are many of them that provide acceptable results. These operators use the aforementioned two kernels (typically of size $3\times3$ or $5\times5$) which are convolved with the original image to calculate approximations of the derivatives. 

These are some examples (first column: operator name; second one: $H_R$; third column: $H_C$):$\\[5pt]$

<div style="padding: 10px; margin: 0 auto; text-align: center;">

<div style = "padding: 0px; display: table-cell; vertical-align: middle; width: 70px; text-align:left;">Roberts</div>
    
<div style = "padding: 10px; display: table-cell; vertical-align: middle; width: 50px; text-align:right;"> </div>
    
<table style = "display: table-cell; vertical-align: middle;">
	<tbody>
		<tr>
			<td style="border: 1px solid;">0</td>
			<td style="border: 1px solid;">0</td>
			<td style="border: 1px solid;">0</td>
		</tr>
		<tr>
			<td style="border: 1px solid;">0</td>
			<td style="border: 1px solid;" bgcolor="#81F7F3">0</td>
			<td style="border: 1px solid;">1</td>
		</tr>
		<tr>
			<td style="border: 1px solid;">0</td>
			<td style="border: 1px solid;">-1</td>
			<td style="border: 1px solid;">0</td>
		</tr>
	</tbody>
</table>   
    
<div style = "padding: 10px; display: table-cell; vertical-align: middle; width: 50px; text-align:right;"> </div>
    
<table style = "display: table-cell; vertical-align: middle;">
	<tbody>
		<tr>
			<td style="border: 1px solid;">-1</td>
			<td style="border: 1px solid;">0</td>
			<td style="border: 1px solid;">0</td>
		</tr>
		<tr>
			<td style="border: 1px solid;">0</td>
			<td style="border: 1px solid;" bgcolor="#81F7F3">1</td>
			<td style="border: 1px solid;">0</td>
		</tr>
		<tr>
			<td style="border: 1px solid;">0</td>
			<td style="border: 1px solid;">0</td>
			<td style="border: 1px solid;">0</td>
		</tr>
	</tbody>
</table>  

</div>

<div style="padding: 10px; margin: 0 auto; text-align: center;">

<div style = "padding: 0px; display: table-cell; vertical-align: middle; width: 70px; text-align:left;">Prewitt</div>
    
<div style = "padding: 10px; display: table-cell; vertical-align: middle; width: 50px; text-align:right;">$\frac{1}{3}$ </div>
    
<table style = "display: table-cell; vertical-align: middle;">
	<tbody>
		<tr>
			<td style="border: 1px solid;">1</td>
			<td style="border: 1px solid;">0</td>
			<td style="border: 1px solid;">-1</td>
		</tr>
		<tr>
			<td style="border: 1px solid;">1</td>
			<td style="border: 1px solid;" bgcolor="#81F7F3">0</td>
			<td style="border: 1px solid;">-1</td>
		</tr>
		<tr>
			<td style="border: 1px solid;">1</td>
			<td style="border: 1px solid;">0</td>
			<td style="border: 1px solid;">-1</td>
		</tr>
	</tbody>
</table>   
    
<div style = "padding: 5px; display: table-cell; vertical-align: middle; width: 50px; text-align:right;">$\frac{1}{3}$ </div>
    
<table style = "display: table-cell; vertical-align: middle;">
	<tbody>
		<tr>
			<td style="border: 1px solid;">-1</td>
			<td style="border: 1px solid;">-1</td>
			<td style="border: 1px solid;">-1</td>
		</tr>
		<tr>
			<td style="border: 1px solid;">0</td>
			<td style="border: 1px solid;" bgcolor="#81F7F3">0</td>
			<td style="border: 1px solid;">0</td>
		</tr>
		<tr>
			<td style="border: 1px solid;">1</td>
			<td style="border: 1px solid;">1</td>
			<td style="border: 1px solid;">1</td>
		</tr>
	</tbody>
</table>  

</div>

<div style="padding: 10px; margin: 0 auto; text-align: center;">

<div style = "padding: 0px; display: table-cell; vertical-align: middle; width: 70px; text-align:left;">Sobel</div>
    
<div style = "padding: 10px; display: table-cell; vertical-align: middle; width: 50px; text-align:right;">$\frac{1}{4}$ </div>
    
<table style = "display: table-cell; vertical-align: middle;">
	<tbody>
		<tr>
			<td style="border: 1px solid;">1</td>
			<td style="border: 1px solid;">0</td>
			<td style="border: 1px solid;">-1</td>
		</tr>
		<tr>
			<td style="border: 1px solid;">2</td>
			<td style="border: 1px solid;" bgcolor="#81F7F3">0</td>
			<td style="border: 1px solid;">-2</td>
		</tr>
		<tr>
			<td style="border: 1px solid;">1</td>
			<td style="border: 1px solid;">0</td>
			<td style="border: 1px solid;">-1</td>
		</tr>
	</tbody>
</table>   
    
<div style = "padding: 5px; display: table-cell; vertical-align: middle; width: 50px; text-align:right;">$\frac{1}{4}$ </div>
    
<table style = "display: table-cell; vertical-align: middle;">
	<tbody>
		<tr>
			<td style="border: 1px solid;">-1</td>
			<td style="border: 1px solid;">-2</td>
			<td style="border: 1px solid;">-1</td>
		</tr>
		<tr>
			<td style="border: 1px solid;">0</td>
			<td style="border: 1px solid;" bgcolor="#81F7F3">0</td>
			<td style="border: 1px solid;">0</td>
		</tr>
		<tr>
			<td style="border: 1px solid;">1</td>
			<td style="border: 1px solid;">2</td>
			<td style="border: 1px solid;">1</td>
		</tr>
	</tbody>
</table>  

</div>

<div style="padding: 10px; margin: 0 auto; text-align: center;">

<div style = "padding: 0px; display: table-cell; vertical-align: middle; width: 70px; text-align:left;">Frei-Chen</div>
    
<div style = "padding: 10px; display: table-cell; vertical-align: middle; width: 70px; text-align:right;">$\frac{1}{2+\sqrt{2}}$ </div>
    
<table style = "display: table-cell; vertical-align: middle;">
	<tbody>
		<tr>
			<td style="border: 1px solid;">1</td>
			<td style="border: 1px solid;">0</td>
			<td style="border: 1px solid;">-1</td>
		</tr>
		<tr>
			<td style="border: 1px solid;">$\sqrt{2}$</td>
			<td style="border: 1px solid;" bgcolor="#81F7F3">0</td>
			<td style="border: 1px solid;">-$\sqrt{2}$</td>
		</tr>
		<tr>
			<td style="border: 1px solid;">1</td>
			<td style="border: 1px solid;">0</td>
			<td style="border: 1px solid;">-1</td>
		</tr>
	</tbody>
</table>   
    
<div style = "padding: 10px; display: table-cell; vertical-align: middle; width: 70px; text-align:right;">$\frac{1}{2+\sqrt{2}}$ </div>
    
<table style = "display: table-cell; vertical-align: middle;">
	<tbody>
		<tr>
			<td style="border: 1px solid;">-1</td>
			<td style="border: 1px solid;">-$\sqrt{2}$</td>
			<td style="border: 1px solid;">-1</td>
		</tr>
		<tr>
			<td style="border: 1px solid;">0</td>
			<td style="border: 1px solid;" bgcolor="#81F7F3">0</td>
			<td style="border: 1px solid;">0</td>
		</tr>
		<tr>
			<td style="border: 1px solid;">1</td>
			<td style="border: 1px solid;">$\sqrt{2}$</td>
			<td style="border: 1px solid;">1</td>
		</tr>
	</tbody>
</table>  

</div>

<div style="padding: 10px; margin: 0 auto; text-align: center;">

<div style = "padding: 0px; display: table-cell; vertical-align: middle; width: 70px; text-align:left;">In general</div>
    
<div style = "padding: 10px; display: table-cell; vertical-align: middle; width: 70px; text-align:right;">$\frac{1}{2+K}$ </div>
    
<table style = "display: table-cell; vertical-align: middle;">
	<tbody>
		<tr>
			<td style="border: 1px solid;">1</td>
			<td style="border: 1px solid;">0</td>
			<td style="border: 1px solid;">-1</td>
		</tr>
		<tr>
			<td style="border: 1px solid;">$K$</td>
			<td style="border: 1px solid;" bgcolor="#81F7F3">0</td>
			<td style="border: 1px solid;">-$K$</td>
		</tr>
		<tr>
			<td style="border: 1px solid;">1</td>
			<td style="border: 1px solid;">0</td>
			<td style="border: 1px solid;">-1</td>
		</tr>
	</tbody>
</table>   
    
<div style = "padding: 10px; display: table-cell; vertical-align: middle; width: 70px; text-align:right;">$\frac{1}{2+K}$ </div>
    
<table style = "display: table-cell; vertical-align: middle;">
	<tbody>
		<tr>
			<td style="border: 1px solid;">-1</td>
			<td style="border: 1px solid;">-$K$</td>
			<td style="border: 1px solid;">-1</td>
		</tr>
		<tr>
			<td style="border: 1px solid;">0</td>
			<td style="border: 1px solid;" bgcolor="#81F7F3">0</td>
			<td style="border: 1px solid;">0</td>
		</tr>
		<tr>
			<td style="border: 1px solid;">1</td>
			<td style="border: 1px solid;">$K$</td>
			<td style="border: 1px solid;">1</td>
		</tr>
	</tbody>
</table>  

</div>

At this point we know how to perform a discrete approximation of a gradient operator through the application of a convolution operation with two different kernels, that is:

$$\nabla  F(x,y) = 
\begin{bmatrix} 
F \otimes H_C \\ F \otimes H_R
\end{bmatrix}$$

But, how could we use the output of those computations to detect edges? The following figure clarifies that!

<img src="./images/how_to_detect_edges_with_gradient_image2.png" width="350">

**Kernel sizes**

As discussed, kernels can be of different size, and that size directly affects the quality of the detection and the localization (e.g. Sobel $3\times3$ or $5\times5$):
- Small template:
  - more precise localization (good localization).
  - more affected by noise (likely produces false positives).
- Large template:
  - less precise localization.
  - more robust to noise (good detector).
  - higher computational cost ($O(N \times N)$)

### **<span style="color:green"><b><i>ASSIGNMENT 2: Playing with Sobe derivatives</i></b></span>**

Now that we have acquired a basic understanding of these methods, let's employ the Sobel kernels ($S_x,S_y$) to compute both derivatives and display them along with the original image (`medical_3.jpg`).

*Notice that the derivative image values can be positive **and negative**, caused by the negative values in the kernel. This implies that the desired depth of the destination image (`ddepth`) has to be at least a signed data type when calling to the `filter2D()` method.*

In [ ]:
# ASSIGNMENT 2
# Read one of the images, compute both kernel derivatives, apply them to the image (separately) and display the original image and the derivatives in a 1x3 plot.
# Write your code here!

# Read the image


# Define horizontal and vertical kernels


# Apply convolution


# And show them!


Once we have computed both derivative images $G_C$ and $G_R$, we can determine the complete edge image by computing the image gradient magnitude and then binarizing the result:

$$|\nabla F(x,y)| = \sqrt{(F \otimes G_C)^2 + (F \otimes G_R)^2} \approx |F \otimes G_C| + |F \otimes G_R|$$

### **<span style="color:green"><b><i>ASSIGNMENT 3a: Time to detect edges</i></b></span>**

Complete `edge_detection_chart()` that computes the gradient image of an input one using `kernel_h` and `kernel_v` (kernels for horizontal and vertical derivatives respectively) and **binarize the resultant image** (final edges image) using `threshold`. Then display in a 1x3 plot `image`, the gradient image, and finally, an image with the detected edges! (Only if `verbose` is True).

*Tip: you should [normalize](https://docs.opencv.org/2.4/modules/core/doc/operations_on_arrays.html#normalize) gradient image before thresholding.*

In [ ]:
# ASSIGNMENT 3a
# Implement a function that that computes the gradient of an image, taking also as input the convolution kernels.
# It must also binarize the resulting image using a threshold
# Show the input image, the gradient image (normalized) and the binarized edge image in a 1x3 plot (if verbose = True)
def edge_detection_chart(image, kernel_h, kernel_v, threshold, verbose=False):
    """ Computed the gradient of the image, binarizes and display it.   
    
        Args:
            image: Input image
            kernel_h: kernel for horizontal derivative
            kernel_v: kernel for vertical derivative
            threshold: threshold value for binarization
            verbose: Only show images if this is True
            
        Returns:
            edges: edges binary image
    """
    
    # Compute derivatives
    
    
    # Compute gradient
    
    
    #Normalize gradient
    
    
    # Threshold to get edges
    
    
    if verbose:
        # Show the initial image
        

        # Show the gradient image
        

        # Show edges image
        


You can use next code to **test if your results are correct**:

In [ ]:
image = np.array([[10,60,20],[60,22,74],[72,132,2]], dtype=np.uint8)

kernel_h = np.array([[1,0,-1],[2,0,-2],[1,0,-1]])*1/4
kernel_v = np.array([[-1,-2,-1],[0,0,0],[1,2,1]])*1/4

print(edge_detection_chart(image, kernel_h, kernel_v, 100))

<font color='blue'>**Expected output:**  </font>

    [[  0   0   0]
     [255 255 255]
     [  0 255   0]]

### **<span style="color:green"><b><i>ASSIGNMENT 3b: Testing our detector</i></b></span>**

Now **try the implemented method** with different size Sobel kernels ($3\times3$, $5\times5$, ...) and with other operators (Roberts, Prewitt, ...). $\\[5pt]$  

In [ ]:
# ASSIGNMENT 3b
# Read the image, set you kernels (Sobel, Roberts, Prewitt, etc.) and interact with the threshold parameter.
# Write your code here!

# Read image
image = cv2.imread(images_path + 'medical_3.jpg', 0)

# Define kernel (Sobel, Roberts, Prewitt, ...)
kernel_h = 
kernel_v = 

#Interact with your code!
interactive( edge_detection_chart, image=fixed(image), kernel_h=fixed(kernel_h), kernel_v=fixed(kernel_v), threshold=(0,255,10), verbose=fixed(True))

### <font color="blue"><b><i>Thinking about it (1)</i></b></font>

Now, **answer following questions**:

- What happens if we use a bigger kernel?
  
    <p style="margin: 4px 0px 6px 5px; color:blue"><i>Your answer here!</i></p>
    
- There are differences between Sobel and other operators?
  
    <p style="margin: 4px 0px 0px 5px; color:blue"><i>Your answer here!</i></p>  
    
- What errors appear using those operators?

    <p style="margin: 4px 0px 0px 5px; color:blue"><i>Your answer here!</i></p>  
    
- Why kernels usually are divided by a number? (e.g. $3\times3$ Sobel is divided by 4)

    <p style="margin: 4px 0px 0px 5px; color:blue"><i>Your answer here!</i></p>  

## 3.1.2 DroG operator

Despite the simplicity of the previous techniques, they have a remarkable drawback: their performance is highly influenced by image noise. Taking a look at the following figure we can see how, having an apparently not so noisy function (first row), where it is easy to detect a step (an abrupt change in its values) around 1000, the response of the derivative with that level of noise is as bigger as the step itself!$\\[5pt]$

<img src="./images/example_of_derivative_of_noisy_function.PNG" width="600"><br />
<center><i>Source: S. Seitz</i></center>$\\[5pt]$

But not everything is lost! An already studied image processing technique can be used to mitigate such noise: **image smoothing**, and more concretely, **Gaussian filtering!**. The basic idea is to smooth the image and then apply a gradient operator, that is to compute $\frac{\partial}{\partial x}(f \otimes g)$. Not only that, this can be done even more efficiently thanks to the convolution derivative property:$\\[5pt]$

$$\frac{\partial}{\partial x}(f \otimes g) = f \otimes \frac{\partial}{\partial x}g
$$$\\[5pt]$

That is, precomputing the resultant kernels from the convolution of the Gaussisan filtering and the Sobel ones, and then convolving them with the image to be processed. With that we save one operation!$\\[5pt]$

This combination of smoothing and gradient is usually called **Derivative of Gaussian operator (DroG)**:$\\[5pt]$

$$
\nabla [f(x,y)\otimes g_\sigma(x,y)] = 
f(x,y) \otimes \nabla [g_\sigma(x,y)] = 
f(x,y) \otimes \text{DroG} (x,y)
$$

$$
DroG(x,y) = 
\nabla \left[ g_\sigma (x,y) \right] = 
\underbrace{\begin{bmatrix} 
\frac{\partial}{\partial x}[g_\sigma(x)g_\sigma(y)] \\ 
\frac{\partial}{\partial y}[g_\sigma(x)g_\sigma(y)] \end{bmatrix}}_{\text{separability}} = 
\underbrace{\begin{bmatrix} \frac{-xg_\sigma(x,y)}{\sigma^2} \\ \frac{-yg_\sigma(x,y)}{\sigma^2}\end{bmatrix}}_{g(x)'=-xg(x)/\sigma^2}
$$

Remember from the previous notebooks the expression of the Gaussian distribution with 2 variables, where the standard deviation $\sigma$ controls the degree of smoothness: 
 
$$g_{\sigma}(x,y) = \frac{1}{2\pi\sigma^2}exp\left(-\frac{x^2+y^2}{2\sigma^2}\right)$$

Take into account that the DroG template or kernel is created just once!

### **<span style="color:green"><b><i>ASSIGNMENT 4: Applying DroG</i></b></span>**

We would like to try this robust edge detection technique, so complete the `gaussian_kernel()` method that constructs a Gaussian filter and then derives it, getting the DroG template. Finally, it calls `edge_detection_chart()` using that template. 

Its inputs are: 
- an image to be processed,
- the kernel aperture size,
- the standard deviation, and
- the gradient image binaritazion threshold.

In [ ]:
# ASSIGNMENT 4
# Implement a function that builds the horizontal and vertical DroG templates and calls to 'edge_detection_chart' using that templates. 
# Inputs: an image, the kernel aperture size, the Gaussian standard deviation and the threshold for the gradient image binarization
# It returns the horizontal and vertical kernels
def gaussian_kernel(image, w_kernel, sigma, threshold, verbose=False):
    """ Construct the DroG operator and call edge_detection_chart.   
    
        Args:
            image: Input image
            w_kernel: Kernel aperture size
            sigma: Standard deviation of the Gaussian distribution
            threshold: Threshold value for binarization
            verbose: Only show images if this is True
            
        Returns:
            kernel_h, kernel_v: DroG horizontal and vertical derivatives
    """
    
    # Create Gaussian filter
    
    
    # Define x and y axis
    
    
    # Construct DroG
    
    
    # Call edge detection chart using DroG


You can use next code to **test if results are correct**:

In [ ]:
# Create an input image
image = np.array([[10,60,20],[60,22,74],[72,132,2]], dtype=np.uint8)

# Apply the Gaussian kernel
gaussian_kernel(image, w_kernel=1, sigma=1.2, threshold=100)

<font color='blue'>**Expected output:**  </font>

    (array([[ 0.03832673, -0.        , -0.03832673],
            [ 0.05423735, -0.        , -0.05423735],
            [ 0.03832673, -0.        , -0.03832673]]),
     array([[ 0.03832673,  0.05423735,  0.03832673],
            [-0.        , -0.        , -0.        ],
            [-0.03832673, -0.05423735, -0.03832673]]))

### <font color="blue"><b><i>Thinking about it (2)</i></b></font>

Now **try this method** and play with interactive parameters. Then **answer the following questions**:

- What happens if a bigger kernel is used?
  
    <p style="margin: 4px 0px 6px 5px; color:blue"><i>Your answer here!</i></p>
    
- What kind of errors appear and disappear whenever sigma is modified?
  
    <p style="margin: 4px 0px 0px 5px; color:blue"><i>Your answer here!</i></p>   
    
- Why the gradient image have lower values than the one from the original image? *Tip: [image normalization](https://stackoverflow.com/questions/40645985/opencv-python-normalize-image/42164670)* 

    <p style="margin: 4px 0px 0px 5px; color:blue"><i>Your answer here!</i></p>   
    
- Now that you have tried different techniques, in your opinion, which is the best one for this type of images?

    <p style="margin: 4px 0px 0px 5px; color:blue"><i>Your answer here!</i></p>   

In [ ]:
# Read the image
image = cv2.imread(images_path + 'medical_3.jpg', 0)

# Interact with the three input parameters
interactive(gaussian_kernel, image=fixed(image), w_kernel=(1,5,1), sigma=(0.4,5,0.5), threshold=(0,255,10), verbose=fixed(True))

### **<span style="color:green"><b><i>ASSIGNMENT 5: Measuring efficiency</i></b></span>**

Finally, **you are asked to** compare the execution time of creating a DroG template using your previous code (combining two 1-D gaussian filters), with creating a DroG template using a 2-D gaussian filter. *Use a great aperture size (e.g 500)*.

*Tip: [how to measure execution time in Python](https://stackoverflow.com/questions/14452145/how-to-measure-time-taken-between-lines-of-code-in-python)*

In [ ]:
# ASSIGNMENT 5

import time

print("Measuring the execution time needed for ...")
      
# Start timer

# Create 1D Gaussian filter


# Define x and y axis


# Construct DroG using 1D Gaussian filters


# Stop timer


# Start timer


# Create 2D Gaussian filter


# Define x and y axis


# Construct DroG using 2D Gaussian filter


# Stop timer

## Conclusion

Awesome! Now you have expertise in more applications of the convolution operator. In this notebook you:

- Learned basic operators for edge detection that perform a **discrete approximation of a gradient operator**.
- Learned **how to construct a DroG kernel** in an efficient way.
- Layed a bit with them in the context of medical images, discovering some real and important utilities.